In [1]:
# importing libraries
import torch
import torch.nn as nn
import math
from typing import Tuple, Optional

d:\Projects\Coding\Python\llama-from-scratch\venv\Lib\site-packages\torch\_subclasses\functional_tensor.py:276: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


# Config settings

#### Architecural Dimensions
$\underline{\text{Grouped-Query Attention (GQA)}}$  
This is a technique in which we use fewer number of Key/Value heads than the Query heads.  This method requires significantly less memory, and can generate text much faster with a very small impact on the overall accuracy.  
> __note:__ The number of Query heads must be perfectly divisble by the number of Key/Value heads.  

*In this case, we use one Key and Value heads per 4 Query heads*

In [2]:

hidden_size = 128   # synonymous with embeddings dimension
num_attention_heads = 16    # The no. of attention query heads
num_key_value_heads = 4     # The no. of Key & Value heads [Grouped-Query Attention (GQA)]
head_dim = hidden_size // num_attention_heads   # Dimension of each atention head

#### Positional Embedding Parameters

Instead of traditional positional embeddings, this model uses RoPE to encode the order of tokens. RoPE modifies the Query and Key vectors using rotations, which elegantly injects relative positional information directly into the self-attention calculation.  


$\underline{\text{Rotary Positional Encodings (RoPE)}}$  
* **Relative Position:** The attention score between two tokens becomes sensitive to their relative distance, not their absolute positions.
* **No Trainable Parameters:** Positional information is added via a deterministic function, requiring no extra parameters to be learned.
* **Long Sequence Extrapolation:** RoPE has been shown to be effective at handling sequences longer than the model was trained on.

In [3]:
max_positional_embeddings = 256  # max no. of positions to be calculated by RoPE\
rope_theta = 10000  # base for the formula to calculate frequencies for RoPE, controlling the timescale

#### Normalization and Regularization

In [4]:
rms_norm_eps = 1e-5 # to normalise the vector embeddings
attention_bias = 0  # 0 to keep it as a Linear Layer without an extra bias vector
attention_dropout = 0  # Dropout probability for attention weights to prevent overfitting, for simplicity, we won't use that
use_qk_norm = True  # To apply L2 normalization on Q & K before attention

#### Sample Input

In [5]:
torch.manual_seed(77)  # for reproducibility

In [6]:
batch_size = 2  # two independent sequences of text
sequence_length = 10  # length of each sequence

hidden_states = torch.randn(batch_size, sequence_length, hidden_size)  # creating sample input token embeddings

In [7]:
print(hidden_states)
print(hidden_states.shape)

tensor([[[-0.5043, -0.4161, -0.1364,  ..., -1.5856, -0.4089, -2.8163],
         [ 1.0667, -0.0923,  0.3463,  ...,  0.5123,  1.9678, -1.6733],
         [ 1.2775,  0.2651, -0.5682,  ..., -0.2129, -1.4258, -1.2878],
         ...,
         [ 1.4049, -0.0547, -0.4749,  ...,  2.6301, -0.4774,  0.3909],
         [-0.5966,  0.7187, -0.3401,  ..., -0.5780,  0.9983,  0.6903],
         [-0.4571,  0.7204,  0.3816,  ...,  1.9020, -0.6863,  0.4856]],

        [[-1.8869,  2.0450, -0.3714,  ..., -0.0561,  1.2780, -0.0363],
         [ 0.2985,  1.5429,  1.3085,  ...,  0.2492,  0.6134,  0.5383],
         [-0.2063, -2.8666, -1.4368,  ..., -0.6156, -0.6485,  0.1808],
         ...,
         [-0.2064,  2.1962,  1.2381,  ...,  1.1080, -0.6104,  0.7092],
         [-1.1046, -0.1936,  0.0943,  ..., -0.0681,  0.0745,  1.0041],
         [ 0.8959,  0.1819,  1.3658,  ...,  0.7530, -0.9845, -0.2993]]])
torch.Size([2, 10, 128])


The job of `position_ids` is to tell the RoPE function the position of each token (is it the 1st, 2nd, 3rd, etc. token?).

In [8]:
# to create positional ids for the tokens (very imp for RoPE)
position_ids = torch.arange(0, sequence_length).unsqueeze(0).repeat(batch_size, 1)

In [9]:
# understanding what goes on in each step
print(f'torch.arange(0, sequence_length) : \n {torch.arange(0, sequence_length)}')  # creates a simple indexing sequence of numbers
print()
print(f'torch.arange(0, sequence_length).unsqueeze(0) : \n {torch.arange(0, sequence_length).unsqueeze(0)}')  # Adds a new dimension of size 1 at the specified position (0-row, 1-column)
print()
print(f'torch.arange(0, sequence_length).unsqueeze(0).repeat(batch_size, 1) : \n {torch.arange(0, sequence_length).unsqueeze(0).repeat(batch_size, 1)}')  # copies the sequence for each item in the batch

torch.arange(0, sequence_length) : 
 tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

torch.arange(0, sequence_length).unsqueeze(0) : 
 tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

torch.arange(0, sequence_length).unsqueeze(0).repeat(batch_size, 1) : 
 tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])


#### Attention Mask
To prevent the model from attending to the tokens after the current token, it should be able to see only the previous tokens.

In [10]:
# create a square and assign -inf to all the upper triangle positions so the softmax functino will make it 0
# diagonal = 1 specifies that the digonal right above the principal diagonal
attention_mask = torch.triu(torch.ones(sequence_length, sequence_length) * -torch.inf, diagonal=1)
print(attention_mask)
print(attention_mask.shape)

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
torch.Size([10, 10])


* `.unsqueeze(0)` changes the dimension from `[sequence_length, sequence_length]` to `[1, sequence_length, sequence_length]`  

* second `.unsqueeze(0)` changes the dimension from `[1, sequence_length, sequence_length]` to `[1, 1, sequence_length, sequence_length]`   

We do this to match the dimensions with the attention_weights, which has a 4D shape `[batch_size, num_attention_heads, sequence_length, sequence_length]`

In [11]:
attention_mask =attention_mask.unsqueeze(0).unsqueeze(0)
print(attention_mask)
print(attention_mask.shape)

tensor([[[[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
          [0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., -inf],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]])
torch.Size([1, 1, 10, 10])


* `1` indicates that the size of the second dimension is 1. We apply the same attention mask across all attention heads. 
* `-1` indicates to not change the third and fourth dimensions, to remain unchanged

In [12]:
attention_mask = attention_mask.expand(batch_size, 1, -1, -1)
print(attention_mask)
print(attention_mask.shape)

tensor([[[[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
          [0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., -inf],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],


        [[[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -in

### Final Config Check

In [13]:
print('Configuration')
print(f'hidden_size: {hidden_size}')
print(f'num_attention_heads: {num_attention_heads}')
print(f'num_key_value_heads: {num_key_value_heads}')
print(f'head_dim: {head_dim}')
print()
print('Sample Input Shapes')
print(f'hidden_states: {hidden_states.shape}')
print(f'position_ids: {position_ids.shape}')
print(f'attention_mask: {attention_mask.shape}')

Configuration
hidden_size: 128
num_attention_heads: 16
num_key_value_heads: 4
head_dim: 8

Sample Input Shapes
hidden_states: torch.Size([2, 10, 128])
position_ids: torch.Size([2, 10])
attention_mask: torch.Size([2, 1, 10, 10])


# Q, K, V Projections

#### Define Projection Layers
here, we define the matrices W<sup>q</sup>, W<sup>k</sup>, and W<sup>v</sup>

In [14]:
q_proj = nn.Linear(hidden_size, num_attention_heads * head_dim, bias= attention_bias)
k_proj = nn.Linear(hidden_size, num_key_value_heads * head_dim, bias= attention_bias)
v_proj = nn.Linear(hidden_size, num_key_value_heads * head_dim, bias= attention_bias)

# contains the learned weight matrix often refferd to as W_o on paper
o_proj = nn.Linear(num_attention_heads * head_dim, hidden_size, bias= attention_bias)

In [15]:
print('Projection layers')
print(f'q_proj: {q_proj}')
print(f'k_proj: {k_proj}')
print(f'v_proj: {v_proj}')
print(f'o_proj: {o_proj}')

Projection layers
q_proj: Linear(in_features=128, out_features=128, bias=False)
k_proj: Linear(in_features=128, out_features=32, bias=False)
v_proj: Linear(in_features=128, out_features=32, bias=False)
o_proj: Linear(in_features=128, out_features=128, bias=False)


#### Project the input on these matrices

In [16]:
query_states = q_proj(hidden_states)
key_states = k_proj(hidden_states)
value_states = v_proj(hidden_states)

In [17]:
print('Projections shape')
print(f'query_states: {query_states.shape}')
print(f'key_states: {key_states.shape}')
print(f'value_states: {value_states.shape}')

Projections shape
query_states: torch.Size([2, 10, 128])
key_states: torch.Size([2, 10, 32])
value_states: torch.Size([2, 10, 32])


* Query: We have 2 sequences, 10 tokens in each sequence, and 128 values to represent each single token.  
* Key & Value: We have 2 sequences, 10 tokens in each sequence, and 32 values to represent each single token.

#### Creating Individual Heads
We have have Q, K, and V. We must divide them into individual heads for multi-head attention.  

Target shape: [batch_size, num_heads, sequence_length, head_size]

`view()` function is used to reshape the tensor *(works only on contiguous tensors)*  
*we used it to split `hidden_size` dimension into two new dimensions `(num_attention_heads, head_dim)`*

In [18]:
query_states = query_states.view(batch_size, sequence_length, num_attention_heads, head_dim).transpose(1,2)
key_states = key_states.view(batch_size, sequence_length, num_key_value_heads, head_dim).transpose(1,2)
value_states = value_states.view(batch_size, sequence_length, num_key_value_heads, head_dim).transpose(1,2)

In [19]:
print('The individual heads shapes:')
print(f'query_states: {query_states.shape}')
print(f'key_states: {key_states.shape}')
print(f'value_states: {value_states.shape}')

The individual heads shapes:
query_states: torch.Size([2, 16, 10, 8])
key_states: torch.Size([2, 4, 10, 8])
value_states: torch.Size([2, 4, 10, 8])


#### Calculating the number of Query heads per Key-Value head

In [20]:
num_key_value_groups = num_attention_heads // num_key_value_heads
print(f'Number of Key-Value groups (Q heads per K-V head): {num_key_value_groups}')

Number of Key-Value groups (Q heads per K-V head): 4


# Rotary Position Embeddings (RoPE)

#### Defining Rotation Calculation Function - `simple_rope_calculation()`

In [21]:
def simple_rope_calculation(dim, max_seq_len, base=10000.0, device=None):
    
    # Please find the breakdown of whats happening in each line below in the next cell

    inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2, device=device).float() / dim))  # LINE 1
    t = torch.arange(max_seq_len, device=device).type_as(inv_freq)  # LINE 2
    freqs = torch.outer(t, inv_freq)  # LINE 3
    emb = torch.cat((freqs, freqs), dim=1)  # LINE 4 
    
    # To create rotators for "cos(theta) + i*sin(theta)"
    freqs_cos = emb.cos()  # real part
    freqs_sin = emb.sin()  # imaginery part
    freqs_cis = torch.complex(freqs_cos, freqs_sin)  # LINE 5

    return freqs_cis
    

#### Breakdown of `simple_rope_calculation()`

In [22]:
dim = 8
device = None
base=10000.0
max_seq_len = 10
print(f'variables- dim:{dim}, device:{device}, base:{base}, max_seq_len:{max_seq_len}')

variables- dim:8, device:None, base:10000.0, max_seq_len:10


LINE 1

In [23]:
print('\nCreate a vector')
print(torch.arange(0, dim, 2, device=device).float())

# To normalise, we divide by the dimension
print('\nTo normalise, we divide by the dimension')
print(torch.arange(0, dim, 2, device=device).float() / dim)

# raising base to the power
print('\nRaising base to the power of the normalized vector')
print(base ** (torch.arange(0, dim, 2, device=device).float() / dim))

# divide 1 by all these
print('\nDivide 1 by all these (reciprocal)')
print(1.0 / (base ** (torch.arange(0, dim, 2, device=device).float() / dim)))

# full line
inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2, device=device).float() / dim))


Create a vector
tensor([0., 2., 4., 6.])

To normalise, we divide by the dimension
tensor([0.0000, 0.2500, 0.5000, 0.7500])

Raising base to the power of the normalized vector
tensor([   1.,   10.,  100., 1000.])

Divide 1 by all these (reciprocal)
tensor([1.0000, 0.1000, 0.0100, 0.0010])


LINE 2

In [24]:
t = torch.arange(max_seq_len, device=device).type_as(inv_freq)
print(f'\'t\' shows the position for each token: {t}')

't' shows the position for each token: tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])


LINE 3  

We find the outer product of `k` and `inv_freq`

In [25]:
freqs = torch.outer(t, inv_freq)
print(freqs)
print(freqs.shape)

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.0000e+00, 1.0000e-01, 1.0000e-02, 1.0000e-03],
        [2.0000e+00, 2.0000e-01, 2.0000e-02, 2.0000e-03],
        [3.0000e+00, 3.0000e-01, 3.0000e-02, 3.0000e-03],
        [4.0000e+00, 4.0000e-01, 4.0000e-02, 4.0000e-03],
        [5.0000e+00, 5.0000e-01, 5.0000e-02, 5.0000e-03],
        [6.0000e+00, 6.0000e-01, 6.0000e-02, 6.0000e-03],
        [7.0000e+00, 7.0000e-01, 7.0000e-02, 7.0000e-03],
        [8.0000e+00, 8.0000e-01, 8.0000e-02, 8.0000e-03],
        [9.0000e+00, 9.0000e-01, 9.0000e-02, 9.0000e-03]])
torch.Size([10, 4])


Rows represents the tokens, and the columns represents the pairs of embeddings for each token.  
*Example:*  
* *First token: (No rotation ) 0.0000e+00*
* *The second token:*
    * *first pair will rotate by 1.0000e+00*
    * *Second pair will rotate by 1.0000e-01*
    * ...

LINE 4  

We calulated only 4 frequencies for the rotations, but in reality we have `dim=8` embeddings so we need 8 angles. Hence, we achieve this by replicating the `freqs` matrix

In [26]:
emb = torch.cat((freqs, freqs), dim=1)
print(emb.shape)

torch.Size([10, 8])


LINE 5  

`freqs_cis` frequencies in the form of cosine + i*sine

In [27]:
# To create rotators for "cos(theta) + i*sin(theta)"
freqs_cos = emb.cos()  # real part
freqs_sin = emb.sin()  # imaginery part
freqs_cis = torch.complex(freqs_cos, freqs_sin)
print(freqs_cis)
print(freqs_cis.shape)

tensor([[ 1.0000+0.0000j,  1.0000+0.0000j,  1.0000+0.0000j,  1.0000+0.0000j,
          1.0000+0.0000j,  1.0000+0.0000j,  1.0000+0.0000j,  1.0000+0.0000j],
        [ 0.5403+0.8415j,  0.9950+0.0998j,  0.9999+0.0100j,  1.0000+0.0010j,
          0.5403+0.8415j,  0.9950+0.0998j,  0.9999+0.0100j,  1.0000+0.0010j],
        [-0.4161+0.9093j,  0.9801+0.1987j,  0.9998+0.0200j,  1.0000+0.0020j,
         -0.4161+0.9093j,  0.9801+0.1987j,  0.9998+0.0200j,  1.0000+0.0020j],
        [-0.9900+0.1411j,  0.9553+0.2955j,  0.9996+0.0300j,  1.0000+0.0030j,
         -0.9900+0.1411j,  0.9553+0.2955j,  0.9996+0.0300j,  1.0000+0.0030j],
        [-0.6536-0.7568j,  0.9211+0.3894j,  0.9992+0.0400j,  1.0000+0.0040j,
         -0.6536-0.7568j,  0.9211+0.3894j,  0.9992+0.0400j,  1.0000+0.0040j],
        [ 0.2837-0.9589j,  0.8776+0.4794j,  0.9988+0.0500j,  1.0000+0.0050j,
          0.2837-0.9589j,  0.8776+0.4794j,  0.9988+0.0500j,  1.0000+0.0050j],
        [ 0.9602-0.2794j,  0.8253+0.5646j,  0.9982+0.0600j,  1.0000+0.

#### Defining Rotations Function - `apply_rotary_emb_torch()`

In [28]:
def apply_rotary_emb_torch(xq: torch.Tensor, xk: torch.Tensor, freqs_cis: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
    
    # making sure "freqs_cis" is on the right device
    freqs_cis = freqs_cis.to(xq.device)

    # intead of having all the frequencies, we filter out the ones for the correspondings tokens we need.  
    freqs_cis = freqs_cis[position_ids]  # LINE 6

    freqs_cis = freqs_cis[:, None, :, :]  # LINE 7

    xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2))  # LINE 8
    xk_ = torch.view_as_complex(xk.float().reshape(*xk.shape[:-1], -1, 2))
    
    freqs_cis_broadcast = freqs_cis[..., :xq_.shape[-1]]

    # Applying the rotations
    xq_rotated = xq_ * freqs_cis_broadcast
    xk_rotated = xk_ * freqs_cis_broadcast

    # Convert back to real representatinos
    # This flattens the last 2 dimensions back into one.
    # input: [batch, num_heads, seq_length, head_dim / 2 (4), 2] (complex)
    # output: [batch, num_heads, seq_length, 8]
    xq_out = torch.view_as_real(xq_rotated).flatten(3) 
    xk_out = torch.view_as_real(xk_rotated).flatten(3)

    return xq_out.type_as(xq), xk_out.type_as(xk)


#### Breakdown of `apply_rotary_emb_torch()`

LINE 6

In [29]:
print(f'freqs_cis shape: {freqs_cis.shape}')
print(f'position_ids shape: {position_ids.shape}')
print(f'freqs_cis[position_ids] shape: {freqs_cis[position_ids].shape}')
print(f'final shape: \n{freqs_cis[position_ids][0]}')

freqs_cis shape: torch.Size([10, 8])
position_ids shape: torch.Size([2, 10])
freqs_cis[position_ids] shape: torch.Size([2, 10, 8])
final shape: 
tensor([[ 1.0000+0.0000j,  1.0000+0.0000j,  1.0000+0.0000j,  1.0000+0.0000j,
          1.0000+0.0000j,  1.0000+0.0000j,  1.0000+0.0000j,  1.0000+0.0000j],
        [ 0.5403+0.8415j,  0.9950+0.0998j,  0.9999+0.0100j,  1.0000+0.0010j,
          0.5403+0.8415j,  0.9950+0.0998j,  0.9999+0.0100j,  1.0000+0.0010j],
        [-0.4161+0.9093j,  0.9801+0.1987j,  0.9998+0.0200j,  1.0000+0.0020j,
         -0.4161+0.9093j,  0.9801+0.1987j,  0.9998+0.0200j,  1.0000+0.0020j],
        [-0.9900+0.1411j,  0.9553+0.2955j,  0.9996+0.0300j,  1.0000+0.0030j,
         -0.9900+0.1411j,  0.9553+0.2955j,  0.9996+0.0300j,  1.0000+0.0030j],
        [-0.6536-0.7568j,  0.9211+0.3894j,  0.9992+0.0400j,  1.0000+0.0040j,
         -0.6536-0.7568j,  0.9211+0.3894j,  0.9992+0.0400j,  1.0000+0.0040j],
        [ 0.2837-0.9589j,  0.8776+0.4794j,  0.9988+0.0500j,  1.0000+0.0050j,
   

LINE 7
  
The Query and Key tensors have a shape of `[batch, num_heads, seq_len, head_dim]`. So, we add a dimension of `1` to `freqs_cis` to align with Q & K so we can broadcast them with this angles tensor.  

Can use `unsqueeze(1)` too

In [30]:
freqs_cis = freqs_cis[position_ids]
print(freqs_cis.shape)
freqs_cis = freqs_cis[:, None, :, :]
print(freqs_cis.shape)

torch.Size([2, 10, 8])
torch.Size([2, 1, 10, 8])


LINE 8  

Now, we reshapre the Query and Key tensors so that the pairs of numbers are treated as complex numbers **(a+ib)**.  
*Same process for "xk"*

In [31]:
# random xq for testing
xq = torch.randn(batch_size, num_attention_heads, sequence_length, head_dim)
xk = torch.randn(batch_size, num_attention_heads, sequence_length, head_dim)
print(xq)
print(f'{xq.shape}\n')


xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2))  # keeps dimensions execpt the last one same, then splits the last one into two
print(xq)
print(xq.shape)

xk_ = torch.view_as_complex(xk.float().reshape(*xk.shape[:-1], -1, 2))  # keeps dimensions execpt the last one same, then splits the last one into two


tensor([[[[ 0.8011, -0.5517,  1.2441,  ..., -0.8717, -0.2942,  0.3337],
          [ 0.1762, -0.1274, -0.8008,  ..., -1.1549, -0.4158, -1.7951],
          [-2.3783, -0.9613, -2.1015,  ...,  1.3007, -1.0858,  0.3678],
          ...,
          [-0.5342,  0.1280, -0.4700,  ...,  0.9925,  0.6600,  1.5047],
          [ 0.6374,  1.0656,  1.0083,  ...,  1.0599,  1.4468, -0.8652],
          [ 0.0680,  0.1107, -1.6959,  ...,  0.9235,  1.0702,  0.6899]],

         [[ 1.5018,  0.9565, -1.0583,  ...,  0.8433, -0.3213,  1.2631],
          [ 0.7975, -0.1302, -0.2862,  ...,  1.8829,  0.0470,  1.5126],
          [-0.1961,  0.0146,  0.3894,  ..., -0.9640,  0.0291, -0.8071],
          ...,
          [-0.4551, -0.5375,  1.8866,  ...,  0.1935, -0.1347, -0.1596],
          [ 1.0800, -2.1215, -1.3935,  ..., -0.1580, -0.9938, -1.1936],
          [ 0.4865,  0.2801, -0.1601,  ...,  0.4223,  0.5414,  0.1586]],

         [[ 0.5211,  0.2147,  1.4402,  ...,  1.8647, -1.8323,  1.3069],
          [ 0.3001,  0.2398, -

*8 has changed to 4 because we divided the deimension into pairs*

LINE 9  

Previsously, we calculated the `freqs_cis` tensor by using `head_dim` sines and cosines. However, xq's last dimension is now 4 (*Because each pair of numbers become a single complex number*).  
To multiply them, we need freq_cis in the same dimesnions too.

In [32]:
print(f'initially: {freqs_cis.shape}')

freqs_cis_broadcast = freqs_cis[..., :xq_.shape[-1]]
print(freqs_cis_broadcast)
print(freqs_cis_broadcast.shape)

initially: torch.Size([2, 1, 10, 8])
tensor([[[[ 1.0000+0.0000j,  1.0000+0.0000j,  1.0000+0.0000j,  1.0000+0.0000j],
          [ 0.5403+0.8415j,  0.9950+0.0998j,  0.9999+0.0100j,  1.0000+0.0010j],
          [-0.4161+0.9093j,  0.9801+0.1987j,  0.9998+0.0200j,  1.0000+0.0020j],
          [-0.9900+0.1411j,  0.9553+0.2955j,  0.9996+0.0300j,  1.0000+0.0030j],
          [-0.6536-0.7568j,  0.9211+0.3894j,  0.9992+0.0400j,  1.0000+0.0040j],
          [ 0.2837-0.9589j,  0.8776+0.4794j,  0.9988+0.0500j,  1.0000+0.0050j],
          [ 0.9602-0.2794j,  0.8253+0.5646j,  0.9982+0.0600j,  1.0000+0.0060j],
          [ 0.7539+0.6570j,  0.7648+0.6442j,  0.9976+0.0699j,  1.0000+0.0070j],
          [-0.1455+0.9894j,  0.6967+0.7174j,  0.9968+0.0799j,  1.0000+0.0080j],
          [-0.9111+0.4121j,  0.6216+0.7833j,  0.9960+0.0899j,  1.0000+0.0090j]]],


        [[[ 1.0000+0.0000j,  1.0000+0.0000j,  1.0000+0.0000j,  1.0000+0.0000j],
          [ 0.5403+0.8415j,  0.9950+0.0998j,  0.9999+0.0100j,  1.0000+0.0010j],

Now, we have the rotations for 4 pairs in each token.  

**NOTE:** *The slicing step is necessary because we duplicated the frequencies in the `simple_rope_calculation()` function. We do this to understand the existing implementation of transformers.*

#### Calculating RoPE Frequencies

In [33]:
# pre-computed frequencies depending solely on the position, and not the token.
freqs_cis = simple_rope_calculation(head_dim, max_positional_embeddings, base=rope_theta, device=hidden_states.device)
print(f'Calculated freqs_cis shape: {freqs_cis.shape}')

Calculated freqs_cis shape: torch.Size([256, 8])


#### Applying RoPE Rotations

In [34]:
query_states_rope, key_states_rope = apply_rotary_emb_torch(query_states, key_states, freqs_cis)

print('Shapes after RoPE:')
print(f'  query_states_rope: {query_states_rope.shape}')
print(f'  key_states_rope: {key_states_rope.shape}')

Shapes after RoPE:
  query_states_rope: torch.Size([2, 16, 10, 8])
  key_states_rope: torch.Size([2, 4, 10, 8])


# Optional L2 Normalization  
Normalization can sometimes help stabilize the model, espeically for deep neural networks,

In [35]:
class SimpleL2Norm(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.eps = eps # to avoid div by 0 during normalization, we use a v small eps value

    def forward(self, x):
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

In [36]:
if use_qk_norm:
    qk_norm = SimpleL2Norm()
    query_states_final = qk_norm(query_states_rope)
    key_states_final = qk_norm(key_states_rope)
    print('Query & Key states normalised.')
else:
    query_states_final = query_states_rope
    key_states_final = key_states_rope
    print('Query & Key states normalization skipped.')

Query & Key states normalised.


In [37]:
print('Shapes before Attention scores are calculated: ')
print(f'query_states_final: {query_states_final.shape}')
print(f'key_states_final: {key_states_final.shape}')

Shapes before Attention scores are calculated: 
query_states_final: torch.Size([2, 16, 10, 8])
key_states_final: torch.Size([2, 4, 10, 8])


# Grouped Query Attention (GQA)
Since we have fewer K and V heads than Q heads, we need to repeat the K & V heads to match the number of Q heads.

#### Defining Function to repeat the num of attention heads - `Repeat_kv()`}

In [38]:
def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    batch, num_key_value_heads, seq_len, head_dim = hidden_states.shape
    if n_rep == 1:
        return hidden_states
    
    # inserting the new dimension
    hidden_states = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, seq_len, head_dim)

    # replicating the num_heads dimesnsion
    return hidden_states.reshape(batch, num_key_value_heads * n_rep, seq_len, head_dim)

#### Applying the function on K & V states

In [39]:
key_states_repeated = repeat_kv(key_states_final, num_key_value_groups)
value_states_repeated = repeat_kv(value_states, num_key_value_groups)

print('Shapes after repeating K/V for GQA: ')
print(f'key_states_repeated: {key_states_repeated.shape}')
print(f'value_states_repeated: {value_states_repeated.shape}')

Shapes after repeating K/V for GQA: 
key_states_repeated: torch.Size([2, 16, 10, 8])
value_states_repeated: torch.Size([2, 16, 10, 8])


# Attention Scores
Attention scores(q, k, v) = softmax((Q * K^t) / sqrt(dim_of_k/v)) * v  

key -> key^t :  
`(batch, num_attn_heads, seq_len, head_dim) -> (batch, num_attn_heads, head_dim, seq_len)`

In [40]:
attn_weights = torch.matmul(query_states_final, key_states_repeated.transpose(2,3))

In [41]:
# scaling
scaling_factor = 1.0/math.sqrt(head_dim)
attn_weights = attn_weights * scaling_factor # scaled attn weights

print(f'scaling factor: {scaling_factor}')
print(attn_weights)

scaling factor: 0.35355339059327373
tensor([[[[-0.8114, -1.1454,  0.2217,  ..., -0.3674,  0.6383, -1.7795],
          [-1.6664,  1.1880, -0.4083,  ..., -1.4549, -0.9255,  0.4180],
          [-0.4410, -2.3622, -1.4458,  ..., -1.3145,  1.7512, -1.8951],
          ...,
          [-0.9147, -1.7392,  1.1622,  ..., -0.6501,  0.2714, -0.0845],
          [-1.3295,  1.2392,  0.4225,  ..., -1.0408, -0.8553,  1.8397],
          [ 0.3062,  2.0284, -1.2780,  ...,  0.3445, -0.5611,  1.3786]],

         [[-0.2460, -1.8457, -1.1507,  ..., -0.7210,  1.1247, -2.4328],
          [-1.9595, -0.8400,  0.3843,  ..., -1.6351,  0.8147, -0.9490],
          [ 0.7661,  0.9593, -2.0738,  ...,  0.3130,  0.8860,  0.0091],
          ...,
          [ 0.1281,  1.3787,  1.5253,  ...,  0.9533,  0.2300,  1.4844],
          [ 0.3781,  0.0152,  1.5292,  ..., -0.0233, -1.1693,  0.5302],
          [-0.5878,  1.4628,  0.3449,  ..., -0.6731, -1.5125,  1.9564]],

         [[-0.2809,  0.8520,  0.4824,  ..., -0.9812, -2.4452,  1.1

We will apply the attention_mask to the attn_weights to cover the tokens that comes after the current token, this will be called causal mask.  
*Also, to make sure that they're in the same shape*

In [42]:
# Apply the mask
if attention_mask is not None:
    print(f'Applying attention mask with shape: {attention_mask.shape}')

    # Already have the same shape in this notebook but just to be sure
    causal_mask = attention_mask[:, :, :, :key_states_repeated.shape[-2]]
    print(f'\nShape of causal mask: {causal_mask.shape}')

    # adding the mask over the attn_weights
    attn_weights = attn_weights + causal_mask
else:
    print('\nNo Attention mask Applied')

Applying attention mask with shape: torch.Size([2, 1, 10, 10])

Shape of causal mask: torch.Size([2, 1, 10, 10])


shape of `attn_weights`: `[batch_size, num_attention_heads, Q_sequence_length, K_sequence_length]`  

`dim=-1` tells the function to calcualte the softmax probabilities w.r.t the Keys so the query will know how much attention to give each token acc to the attention score assigned to each token based on its key value

In [43]:
attn_weights = nn.functional.softmax(attn_weights, dim=-1).to(query_states.dtype)

print(attn_weights.shape)

torch.Size([2, 16, 10, 10])


#### Dropouts
Basically used to randomly set some attn_weights to 0 to avoid overfitting.  
But, we skip it for simplicity

*   `p=attention_dropout` is set to 0, it's the probability of an element being zeroed out.
*   `training = self.training` tells it to be active only while training, and during eval mode (`model.eval()`) it is set to False

In [44]:
# attn_weights = nn.functional.dropout(attn_weights, p=attention_dropout, training = self.training)

#### Final Attention Value-states Output

In [45]:
attn_output = torch.matmul(attn_weights, value_states_repeated)

In [46]:
print('\nAttention caclulatin Shapes:')
print(f'    attn_weights (raw scores): {attn_weights.shape}')
print(f'    attn_weights (after softmax): {attn_weights.shape}')
print(f'    attn_output: {attn_output.shape}')


Attention caclulatin Shapes:
    attn_weights (raw scores): torch.Size([2, 16, 10, 10])
    attn_weights (after softmax): torch.Size([2, 16, 10, 10])
    attn_output: torch.Size([2, 16, 10, 8])


# Reshaping Output Projections

**How It Works**
**Memory Layout:** After the `.transpose()` and `.contiguous()` operations, your `attn_output` tensor of shape `[2, 10, 16, 8]` is stored in memory as a single, continuous block of `2 * 10 * 16 * 8 = 2560` numbers.

**The view Command:** When you call `.view(2, 10, 128)`, you are giving PyTorch a new blueprint. It takes the flat list of 2560 numbers and starts filling in this new 3D shape:

*   It creates the first dimension of size 2.

*   Inside that, it creates the second dimension of size 10.

*   It then takes the remaining 128 numbers (`16 * 8`) needed to fill the last dimension for each of the `2x10` entries.

Because of the order of the dimensions after the transpose (`[batch, seq_len, heads, head_dim]`), the `heads` and `head_dim` dimensions are next to each other in memory. The `.view()` operation naturally groups them together to form the new `hidden_size` dimension.

In [47]:
# (batch, num_attn_heads, seq_len, head_dim) -> (batch, seq_len, num_attn_heads, head_dim)
attn_output = attn_output.transpose(1,2).contiguous()
print(attn_output.shape)


# -> (batch, seq_len, num_attn_heads * head_dim) = (batch, seq_len, hidden_size)
attn_output = attn_output.view(batch_size, sequence_length, hidden_size)
print(attn_output.shape)

torch.Size([2, 10, 16, 8])
torch.Size([2, 10, 128])


In [48]:
# prijecting it through the LinearLayer we previously created
final_attn_output = o_proj(attn_output)

print('Final Output Shapes:')
print(f'    attn_output (reshaped): {attn_output.shape}')
print(f'    final_attn_output: {final_attn_output.shape}')

Final Output Shapes:
    attn_output (reshaped): torch.Size([2, 10, 128])
    final_attn_output: torch.Size([2, 10, 128])


# Combining it all together

In [52]:
class SimplifiedLlamaAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.hidden_size = config['hidden_size']
        self.num_attention_heads = config['num_attention_heads']
        self.num_key_value_heads = config['num_key_value_heads']
        self.head_dim = self.hidden_size // self.num_attention_heads
        self.num_key_value_groups  =self.num_attention_heads // self.num_key_value_heads
        self.max_positional_embeddings = config['max_positional_embeddings']
        self.rope_theta = config['rope_theta']
        self.attention_bias = config['attention_bias']
        self.use_qk_norm = config['use_qk_norm']

        if (self.head_dim * self.num_attention_heads) != self.hidden_size:
            raise ValueError("hidden_size must be divisible by num_attention_heads")
        
        self.q_proj = nn.Linear(self.hidden_size, self.num_attention_heads * self.head_dim, bias=self.attention_bias)
        self.k_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=self.attention_bias)
        self.v_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=self.attention_bias)
        self.o_proj = nn.Linear(self.num_attention_heads * self.head_dim, self.hidden_size, bias=self.attention_bias)

        self.freqs_cis = simple_rope_calculation(self.head_dim, self.max_positional_embeddings, base=self.rope_theta)
        if self.use_qk_norm:
            self.qk_norm = SimpleL2Norm()
    
    def forward(self, hidden_states, attention_mask, position_ids):
        batch_size, sequence_length, _ = hidden_states.shape

        # projections
        query_states = self.q_proj(hidden_states)
        key_states = self.k_proj(hidden_states)
        value_states = self.v_proj(hidden_states)

        #reshape
        query_states = query_states.view(batch_size, sequence_length, self.num_attention_heads, self.head_dim).transpose(1,2)
        key_states = key_states.view(batch_size, sequence_length, self.num_key_value_heads, self.head_dim).transpose(1,2)
        value_states = value_states.view(batch_size, sequence_length, self.num_key_value_heads, self.head_dim).transpose(1,2)

        # apply rope
        current_freqs_cis = self.freqs_cis.to(hidden_states.device) # getting precomputed freqs on device
        query_states_rope, key_states_rope = apply_rotary_emb_torch(query_states, key_states, current_freqs_cis)

        # optional QK norm
        if self.use_qk_norm:
            query_states_final = self.qk_norm(query_states_rope)
            key_states_final = self.qk_norm(key_states_rope)
        else:
            query_states_final = query_states_rope
            key_states_final = key_states_rope

        # repeater for K/V (GQA)
        key_states_repeated = repeat_kv(key_states_final, self.num_key_value_groups)
        value_states_repeated = repeat_kv(value_states, self.num_key_value_groups)

        # attention calculation
        attn_weights = torch.matmul(query_states_final, key_states_repeated.transpose(2,3))
        scaling_factor = 1.0 / math.sqrt(self.head_dim)
        attn_weights = attn_weights * scaling_factor


        if attention_mask is not None:
            causal_mask = attention_mask[:, :, :, :key_states_repeated.shape[-2]]
            attn_weights = attn_weights + causal_mask

        attn_weights = nn.functional.softmax(attn_weights, dim=-1).to(query_states.dtype)

        # can add dropout if you want

        attn_output = torch.matmul(attn_weights, value_states_repeated)

        # reshape and output projection layer
        attn_output = attn_output.transpose(1,2).contiguous()
        attn_output = attn_output.view(batch_size, sequence_length, self.hidden_size)
        final_attn_output = self.o_proj(attn_output)

        return final_attn_output, attn_weights  # returning weight is optional
    

# Instantiating the simplified model
config_dict = {
    'hidden_size': hidden_size,
    'num_attention_heads': num_attention_heads,
    'num_key_value_heads': num_key_value_heads,
    'max_positional_embeddings': max_positional_embeddings,
    'rope_theta': rope_theta,
    'attention_bias': attention_bias,
    'use_qk_norm': use_qk_norm
}

simplified_attn_module = SimplifiedLlamaAttention(config_dict)

# forward pass
final_output_simplified, final_weights_simplified = simplified_attn_module(hidden_states, attention_mask, position_ids)

print(f"\nOutput shape from simplified attention module: {final_output_simplified.shape}")
print(f'Attention weights shape from simplified attention moduel: {final_weights_simplified.shape}')


Output shape from simplified attention module: torch.Size([2, 10, 128])
Attention weights shape from simplified attention moduel: torch.Size([2, 16, 10, 10])
